In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install fastai==0.7.0
!pip install torchtext==0.2.3
import os

PATH ='data/movie/'

os.makedirs(f'{PATH}',exist_ok=True)
os.makedirs(".kaggle",exist_ok=True)
!pip install -q kaggle
#!mkdir .kaggle

import json 
token = {"username":"","key""}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
 
!chmod 600 /content/.kaggle/kaggle.json

!kaggle config set -n path -v{/content/data}

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle competitions download -c tmdb-box-office-prediction -p /content/'{PATH}'

    100% |████████████████████████████████| 122kB 3.8MB/s 
    100% |████████████████████████████████| 1.5MB 10.2MB/s 
    100% |████████████████████████████████| 3.6MB 10.7MB/s 
    100% |████████████████████████████████| 496.4MB 32kB/s 
    100% |████████████████████████████████| 61kB 21.9MB/s 
    100% |████████████████████████████████| 12.4MB 2.2MB/s 
    100% |████████████████████████████████| 81kB 25.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
  Stored in directory: /root/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
Successfully built bcolz feather-format
mizani 0.5.3 has requirement pandas>=0.23.4, but you'll have pandas 0.22.0 which is incompatible.
plotnine 0.5.1 has requirement pandas>=0.23.4, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.pos

In [0]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
!ls {PATH}

sample_submission.csv  test.csv.zip  train.csv.zip


In [4]:
!unzip /content/data/movie/\*.zip -d data/movie

Archive:  /content/data/movie/test.csv.zip
  inflating: data/movie/test.csv     

Archive:  /content/data/movie/train.csv.zip
  inflating: data/movie/train.csv    

2 archives were successfully processed.


In [0]:
table_names =['train','test']

In [0]:
tables = [pd.read_csv(f'{PATH}{fname}.csv',low_memory=False) for fname in table_names]

In [7]:
for t in tables: display(t.head())

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,3001,"[{'id': 34055, 'name': 'Pokémon Collection', '...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.pokemon.com/us/movies/movie-pokemon...,tt1226251,ja,ディアルガVSパルキアVSダークライ,Ash and friends (this time accompanied by newc...,3.851534,...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Somewhere Between Time & Space... A Legend Is ...,Pokémon: The Rise of Darkrai,"[{'id': 11451, 'name': 'pok√©mon'}, {'id': 115...","[{'cast_id': 3, 'character': 'Tonio', 'credit_...","[{'credit_id': '52fe44e7c3a368484e03d683', 'de..."
1,3002,NaN,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,tt0051380,en,Attack of the 50 Foot Woman,When an abused wife grows to giant size becaus...,3.559789,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,65.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A titanic beauty spreads a macabre wave of hor...,Attack of the 50 Foot Woman,"[{'id': 9748, 'name': 'revenge'}, {'id': 9951,...","[{'cast_id': 2, 'character': 'Nancy Fowler Arc...","[{'credit_id': '55807805c3a3685b1300060b', 'de..."
2,3003,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0118556,en,Addicted to Love,Good-natured astronomer Sam is devastated when...,8.085194,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Comedy About Lost Loves And Last Laughs,Addicted to Love,"[{'id': 931, 'name': 'jealousy'}, {'id': 9673,...","[{'cast_id': 11, 'character': 'Maggie', 'credi...","[{'credit_id': '52fe4330c3a36847f8041367', 'de..."
3,3004,NaN,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",http://www.sonyclassics.com/incendies/,tt1255953,fr,Incendies,A mother's last wishes send twins Jeanne and S...,8.596012,...,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,130.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,The search began at the opening of their mothe...,Incendies,"[{'id': 378, 'name': 'prison'}, {'id': 539, 'n...","[{'cast_id': 6, 'character': 'Nawal', 'credit_...","[{'credit_id': '56478092c3a36826140043af', 'de..."
4,3005,NaN,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",NaN,tt0418753,en,Inside Deep Throat,"In 1972, a seemingly typical shoestring budget...",3.217680,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was filmed in 6 days for 25 thousand dollar...,Inside Deep Throat,"[{'id': 279, 'name': 'usa'}, {'id': 1228, 'nam...","[{'cast_id': 1, 'character': 'Narrator (voice)...","[{'credit_id': '52fe44ce9251416c75041967', 'de..."


In [0]:
train,test=tables

In [9]:
train.belongs_to_collection.value_counts

<bound method IndexOpsMixin.value_counts of 0       [{'id': 313576, 'name': 'Hot Tub Time Machine ...
1       [{'id': 107674, 'name': 'The Princess Diaries ...
2                                                     NaN
3                                                     NaN
4                                                     NaN
5                                                     NaN
6                                                     NaN
7                                                     NaN
8       [{'id': 256377, 'name': 'The Muppet Collection...
9                                                     NaN
10      [{'id': 1575, 'name': 'Rocky Collection', 'pos...
11      [{'id': 48190, 'name': 'Revenge of the Nerds C...
12                                                    NaN
13      [{'id': 91698, 'name': 'Chili Palmer Collectio...
14                                                    NaN
15                                                    NaN
16                          

# Create new feature

In [0]:
import ast

**Add more columns by extracting genres type**

In [0]:
col ='genres'
for table in tables:
    genres_list=[]
    for i in range(len(table[col].values)):    
        try:
            piece_list = ast.literal_eval(table[col].values[i])
            for z in range(len(piece_list)):
                if piece_list[z]['name'] not in genres_list: # modify list name
                    genres_list.append(piece_list[z]['name']) # -> modify list name
                    table[piece_list[z]['name']]=0
                    table.loc[i,piece_list[z]['name']]=1
                else: table.loc[i,piece_list[z]['name']]=1
        except ValueError:
            continue

In [12]:
# transform genres columns to a list of string
'''
for i in range(len(train.genres.values)):
    genres_list=''
    try:
        piece_list = ast.literal_eval(train.genres.values[i])
        for z in range(len(piece_list)):
            genres_list+=piece_list[z]['name']+','
            #genre_type = piece_list[z]['name']
            #train[genre_type]
        train.loc[i,'genres']=genres_list
    except ValueError:
        continue
'''        

"\nfor i in range(len(train.genres.values)):\n    genres_list=''\n    try:\n        piece_list = ast.literal_eval(train.genres.values[i])\n        for z in range(len(piece_list)):\n            genres_list+=piece_list[z]['name']+','\n            #genre_type = piece_list[z]['name']\n            #train[genre_type]\n        train.loc[i,'genres']=genres_list\n    except ValueError:\n        continue\n"

**Add more columns by extracting spoken_languages columns**

In [0]:
col ='spoken_languages'
for table in tables:
    lang_list=[]
    for i in range(len(table[col].values)):    
        try:
            piece_list = ast.literal_eval(table[col].values[i])
            for z in range(len(piece_list)):
                if piece_list[z]['name'] not in lang_list: # modify list name
                    lang_list.append(piece_list[z]['name']) # -> modify list name
                    table[piece_list[z]['name']]=0
                    table.loc[i,piece_list[z]['name']]=1
                else: table.loc[i,piece_list[z]['name']]=1
        except ValueError:
            continue

**Tranform Keywords columns**

In [0]:
col ='Keywords'

for table in tables:    
    for i in range(len(table[col].values)):
        key_list=''
        try:
            piece_list = ast.literal_eval(table[col].values[i])
            for z in range(len(piece_list)): 
                key_list+=piece_list[z]['name']+',' # modify list name
                
            table.loc[i,col]=key_list # modify list name
        except ValueError:
            continue

In [16]:
test['Keywords'].head()

0                            pok√©mon,pocket monsters,
1    revenge,alien,b movie,cheating husband,violenc...
2                      jealousy,love,revenge,break-up,
3    prison,middle east,rape,muslim,militia,interpr...
4         usa,1970s,sexual revolution,unsimulated sex,
Name: Keywords, dtype: object

**production_countries**

In [0]:
col='production_countries'

for table in tables:
    country_list=[]
    for i in range(len(table[col].values)):    
        try:
            piece_list = ast.literal_eval(table[col].values[i])
            for z in range(len(piece_list)):
                if piece_list[z]['name'] not in country_list: # modify list name
                    country_list.append(piece_list[z]['name']) # -> modify list name
                    table[piece_list[z]['name']]=0
                    table.loc[i,piece_list[z]['name']]=1
                else: table.loc[i,piece_list[z]['name']]=1
        except ValueError:
            continue

In [18]:
test[country_list].head()

,Japan,United States of America,Canada,France,Germany,United Kingdom,South Africa,Botswana,China,Sweden,...,Lithuania,United Arab Emirates,Palestinian Territory,Congo,Bolivia,Macedonia,Malaysia,Mauritania,Qatar,Jamaica
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Cast: get gender and name**

In [0]:
col='cast'
'''
#get character name
for table in tables:
    cast_list=[]
    for i in range(len(table[col].values)):    
        try:
            piece_list = ast.literal_eval(table[col].values[i])
            for z in range(len(piece_list)):
                if piece_list[z]['gender'] not in cast_list: # modify list name
                    cast_list.append(piece_list[z]['gender']) # -> modify list name
                    #table[piece_list[z]['gender']]=0
                    #table.loc[i,piece_list[z]['name']]=1
                #else: table.loc[i,piece_list[z]['name']]=1
        except ValueError:
            continue
'''     

In [0]:
#ast.literal_eval(table[col].values[1])

In [0]:
'''
for table in tables:
    
    male = 0
    female = 0
    for i in range(len(table[col].values)):    
        try:
            piece_list = ast.literal_eval(table[col].values[i])
            for z in range(len(piece_list)):
                if piece_list[z]['gender'] == 0: 
                    male +=1
                else: female +=1
                
                table.loc[i,'male_pct']=male/(male+female)
                    
                
        except ValueError:
            continue
'''

**production_companies**

In [0]:
col='production_companies'

for table in tables:
    companies_list=[]
    for i in range(len(table[col].values)):    
        try:
            piece_list = ast.literal_eval(table[col].values[i])
            for z in range(len(piece_list)):
                if piece_list[z]['name'] not in companies_list: # modify list name
                    companies_list.append(piece_list[z]['name']) # -> modify list name
                    table[piece_list[z]['name']]=0
                    table.loc[i,piece_list[z]['name']]=1
                else: table.loc[i,piece_list[z]['name']]=1
        except ValueError:
            continue

In [20]:
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,Chris Lee Productions,Illusion Entertainment,Concorde Pictures,Abbas Kiarostami Productions,Island Pictures,Douglas Fairbanks Pictures,Loofah Productions,Tailor Made,Mango Farms,Quick Six Entertainment
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,0,0,0,0,0,0,0,0,0,0
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,0,0,0,0,0,0,0,0,0,0
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,0,0,0,0,0,0,0,0,0,0
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,0,0,0,0,0,0,0,0,0,0
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,0,0,0,0,0,0,0,0,0,0


**belongs_to_collection**

In [0]:
col='belongs_to_collection'

for table in tables:
    collection_list=[]
    for i in range(len(table[col].values)):    
        try:
            piece_list = ast.literal_eval(table[col].values[i])
            for z in range(len(piece_list)):
                if piece_list[z]['name'] not in collection_list: # modify list name
                    collection_list.append(piece_list[z]['name']) # -> modify list name
                    table[piece_list[z]['name']]=0
                    table.loc[i,piece_list[z]['name']]=1
                else: table.loc[i,piece_list[z]['name']]=1
        except ValueError:
            continue

**release_date**

In [0]:
for table in tables:
    add_datepart(table,'release_date')

#Modeling

In [0]:
for table in tables:
    table = table.drop(['genres','Keywords','spoken_languages','production_companies','production_countries','cast','belongs_to_collection',
                        'homepage','imdb_id','original_title','overview','poster_path','tagline','title','crew',],axis=1)

In [0]:
train = train.drop(['genres','Keywords','spoken_languages','production_companies','production_countries','cast','belongs_to_collection',
                        'homepage','imdb_id','original_title','overview','poster_path','tagline','title','crew',],axis=1)

In [0]:
train_cats(train)

In [40]:
train.shape,test.shape

((3000, 4287), (4398, 5768))

In [48]:
train.head()

,id,budget,original_language,popularity,runtime,status,revenue,Comedy,Drama,Family,...,release_Day,release_Dayofweek,release_Dayofyear,release_Is_month_end,release_Is_month_start,release_Is_quarter_end,release_Is_quarter_start,release_Is_year_end,release_Is_year_start,release_Elapsed
0,1,14000000,en,6.575393,93.0,Released,12314651,1,0,0,...,20,4,51,False,False,False,False,False,False,1424390400
1,2,40000000,en,8.248895,113.0,Released,95149435,1,1,1,...,6,4,219,False,False,False,False,False,False,1091750400
2,3,3300000,en,64.299990,105.0,Released,13092000,0,1,0,...,10,4,283,False,False,False,False,False,False,1412899200
3,4,1200000,hi,3.174936,122.0,Released,16000000,0,1,0,...,9,4,69,False,False,False,False,False,False,1331251200
4,5,0,ko,1.148070,118.0,Released,3923970,0,0,0,...,5,3,36,False,False,False,False,False,False,1233792000


In [0]:
df_train,y,nas = proc_df(train,'revenue')

In [0]:
import sklearn.model_selection as model_selection
X_train,X_valid,y_train,y_valid= model_selection.train_test_split(df_train,y,train_size=0.75,test_size=0.25,random_state=42)

In [46]:
m = RandomForestRegressor(n_jobs=-1,n_estimators=100)
m.fit(X_train,y_train)
m.score(X_train,y_train),m.score(X_valid,y_valid)

(0.9563349651727561, 0.6016374398310727)

In [47]:
m = RandomForestRegressor(n_jobs=-1,n_estimators=100,max_features=0.5)
m.fit(X_train,y_train)
m.score(X_train,y_train),m.score(X_valid,y_valid)

(0.9571472517170281, 0.6603693363769914)